In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


학습할 데이터를 만든다.

In [2]:
xData = [1, 2, 3, 4, 5, 6, 7] # 1일 근무시간
yData = [25000, 55000, 75000, 110000, 128000, 155000, 180000] # 근무시간에 따른 매출금액, 실제값, 레이블, 답, 관측값

과적합을 방지하기 위해서 난수를 발생시켜 기울기(가중치)와 y절편(바이어스)을 정한다.  
과적합이란 학습시킨 데이터에서는 높은 정확도를 보이지만 학습시킨 데이터 이외의 데이터에서는 낮는 정확도를 보이는 문제점을 말한다.

In [3]:
# tf.random_uniform([난수의 개수], 최소값, 최대값): 난수를 발생시킨다.
a = tf.Variable(tf.random_uniform([1], -100, 100)) # 기울기(가중치), -100 ~ 100 사이의 난수
b = tf.Variable(tf.random_uniform([1], -100, 100)) # y절편(바이어스), -100 ~ 100 사이의 난수
print('a = {}, b = {}'.format(a, b))

a = <tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>, b = <tf.Variable 'Variable_1:0' shape=(1,) dtype=float32_ref>


tensorflow 변수를 초기화하고 난수로 발생시킨 기울기와 y절편 값을 확인한다.

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a = {}, b = {}'.format(sess.run(a), sess.run(b)))

a = [-46.904205], b = [54.77838]


근무시간과 매출금액을 기억할 tensorflow에서 사용할 자료형(placeholder)을 선언한다.

In [5]:
x = tf.placeholder(dtype=tf.float32) # 근무시간을 기억할 placeholder
y = tf.placeholder(dtype=tf.float32) # 매출금액을 기억할 placeholder

1차 방정식 형태의 가설을 세우고 오차(비용, 손실) 함수를 정의한다.

In [6]:
# 1차 방정식 형태의 가설, Y => 예측값
Y = a * x + b
# 평균 제곱법을 이용하는 오차(비용, 손실) 함수
# 실제값(yData => y라는 placeholder에 저장한다.)과 예측값(Y)의 편차에 대한 제곱의 평균을 이용한다.
# square(): 제곱값을 계산한다.
# reduce_mean(): 평균을 계산한다.
cost = tf.reduce_mean(tf.square(Y - y))

경사하강법 함수로 오차 함수의 결과를 가장 작게 만드는 방향으로 학습 하도록 정의한다.

In [7]:
# 경사하강 알고리즘의 학습율을 설정한다.
learning_rate = tf.Variable(0.01)
# train.GradientDescentOptimizer(학습율): 학습율에 따른 오차 함수를 계산한다.
# minimize(오차함수): 경사하강 알고리즘에 따라 지정된 오차 함수의 최소값을 찾는다.
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
# train = optimizer.minimize(cost)
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

학습시킨다.

In [8]:
# tensorflow Session을 만들고 변수를 초기화시킨다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(5001):
    # 오차 함수 결과를 가장 작게 만드는 학습을 할 수 있도록 placeholder에 데이터를 대입하고 학습시킨다.
    # 학습할 데이터는 placeholder에 반드시 feed_dict라는 딕셔너리로 대입시켜야 한다.
    # feed_dict의 key에는 placeholder 이름을, value에는 placeholder에 저장할 리스트 또는 튜플 타입의 데이터를 지정한다.
    sess.run(train, feed_dict={x: xData, y: yData})
    
    # 일정한 epoch 단위마다 중간 결과를 출력한다.
    if i % 500 == 0:
        # epoch(i), 오차(cost), 기울기, y절편
        # print('epoch => {:4d}, 손실: {:13.2f} 기울기: {:13.2f} y절편: {:13.2f}'.format(i, 
        #                sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a)[0], sess.run(b)[0]))
        print('epoch => %4d, 손실: %13.2f 기울기: %13.2f y절편: %13.2f' % (i, 
                        sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a), sess.run(b)))

epoch =>    0, 손실: 4598209024.00 기울기:      10382.74 y절편:       1986.87
epoch =>  500, 손실:    8684621.00 기울기:      25539.94 y절편:       1938.20
epoch => 1000, 손실:    8633754.00 기울기:      25627.86 y절편:       1502.84
epoch => 1500, 손실:    8632671.00 기울기:      25640.67 y절편:       1439.40
epoch => 2000, 손실:    8632658.00 기울기:      25642.54 y절편:       1430.15
epoch => 2500, 손실:    8632651.00 기울기:      25642.81 y절편:       1428.81
epoch => 3000, 손실:    8632651.00 기울기:      25642.84 y절편:       1428.63
epoch => 3500, 손실:    8632656.00 기울기:      25642.85 y절편:       1428.61
epoch => 4000, 손실:    8632656.00 기울기:      25642.85 y절편:       1428.61
epoch => 4500, 손실:    8632656.00 기울기:      25642.85 y절편:       1428.61
epoch => 5000, 손실:    8632656.00 기울기:      25642.85 y절편:       1428.61


학습이 완료된 후 원하는 입력에 대한 예측 결과를 출력한다.

In [16]:
for i in range(8, 25):
    print('{:2d}시간 근무를 했을 때 매출 기대금액: {:,.0f}원'.format(i, sess.run(Y, feed_dict={x: [i]})[0]))

 8시간 근무를 했을 때 매출 기대금액: 206,571원
 9시간 근무를 했을 때 매출 기대금액: 232,214원
10시간 근무를 했을 때 매출 기대금액: 257,857원
11시간 근무를 했을 때 매출 기대금액: 283,500원
12시간 근무를 했을 때 매출 기대금액: 309,143원
13시간 근무를 했을 때 매출 기대금액: 334,786원
14시간 근무를 했을 때 매출 기대금액: 360,428원
15시간 근무를 했을 때 매출 기대금액: 386,071원
16시간 근무를 했을 때 매출 기대금액: 411,714원
17시간 근무를 했을 때 매출 기대금액: 437,357원
18시간 근무를 했을 때 매출 기대금액: 463,000원
19시간 근무를 했을 때 매출 기대금액: 488,643원
20시간 근무를 했을 때 매출 기대금액: 514,286원
21시간 근무를 했을 때 매출 기대금액: 539,928원
22시간 근무를 했을 때 매출 기대금액: 565,571원
23시간 근무를 했을 때 매출 기대금액: 591,214원
24시간 근무를 했을 때 매출 기대금액: 616,857원
